In [ ]:
import random
import string
import rtsvg
import re
import shapely.affinity as affinity
rt = rtsvg.RACETrack()

In [2]:
def junk(min_words=16, max_words=128):
    def makeWord(min_chars=3, max_chars=8):
        return ''.join(random.choices(string.ascii_uppercase + string.digits, k=random.randint(min_chars,max_chars)))
    words = []
    for i in range(random.randint(min_words, max_words)):
        words.append(makeWord())
    return ' '.join(words)

my_markers = ['marker1', 'marker2', 'marker3', 'marker4', 'this is a longer marker that is meant to go across lines']
my_colors  = ['#ff0000', '#00ff00', '#0000ff', '#ffff00', '#000000']
all_lus    = {my_markers[0]: my_colors[0], my_markers[1]: my_colors[1], my_markers[2]: my_colors[2], my_markers[3]: my_colors[3], my_markers[4]: my_colors[4]}
my_markups = {'Ex 1': {my_markers[0]: my_colors[0], my_markers[1]: my_colors[1]},
              'Ex 2': {my_markers[2]: my_colors[2], my_markers[0]: my_colors[0]},
              'Ex 3': {my_markers[0]: my_colors[0], my_markers[1]: my_colors[1], my_markers[3]: my_colors[3]},
              'Ex 4': {my_markers[2]: my_colors[2], my_markers[3]: my_colors[3]},
              'Ex 5': {my_markers[4]: my_colors[4]}}
def makePassage(markers):
    _txt_= []
    _txt_.append(junk() + ' ' +random.choice(markers) + ' ' + junk())
    _txt_.append(random.choice(markers) + ' ' + junk(10, 12) + ' ' + random.choice(markers))
    _txt_.append(junk(32,64))
    _txt_.append(junk() + ' ' +random.choice(markers) + ' ' + junk(10, 12) + ' ' + random.choice(markers) + ' ' + junk(10, 12) + ' ' + random.choice(markers))
    _txt_.append(junk(32,256))
    _txt_.append(random.choice(markers) + ' ' + junk(10, 12) + ' ' + random.choice(markers))
    _txt_.append(junk(32,256))
    _txt_.append(random.choice(markers) + ' ' + junk(10, 12) + ' ' + random.choice(markers))
    _txt_.append(junk(32,40))
    _txt_.append(random.choice(markers) + ' ' + junk(10, 12) + ' ' + random.choice(markers))
    return '\n'.join(_txt_)

passage = makePassage(my_markers)
_tb_ = rt.textBlock(passage, word_wrap=True)

In [ ]:
def compareHighlightsPixelReprs(self, 
                                highlights_dict, 
                                y_merge_threshold = 1.0,  # multiple of txt_h
                                y_keep            = 2.0,  # multiple of txt_h, should be smaller than the merge threshold
                                y_render_gap      = 2.0): # multiple of txt_h
    y_merge_threshold *= self.txt_h
    y_keep            *= self.txt_h
    y_render_gap      *= self.txt_h
    # Find all text spans first
    location_lookups = {} # a highlight to the text spans that it covers
    color_lookups    = {}
    for highlighters in highlights_dict:
        highlight_locations = list(highlights_dict[highlighters].keys())
        for highlight_location in highlight_locations:
            color_lookups[highlight_location] = highlights_dict[highlighters][highlight_location]
            if highlight_location not in location_lookups:
                location_lookups[highlight_location] = []
                if   type(highlight_location) is tuple: location_lookups[highlight_location].append(highlight_location) # it's already a span
                elif type(highlight_location) is str:
                    re_match = re.findall(highlight_location,self.txt)
                    if re_match is not None and len(re_match) > 0:
                        for _match in re_match:
                            if type(_match) == tuple: _match = _match[0]
                            i = 0
                            while i < len(self.txt) and _match in self.txt[i:]:
                                i = self.txt.index(_match, i)
                                j = i + len(_match)
                                location_lookups[highlight_location].append((i,j))
                                i = j
                else: raise Exception('unknown highlight type: ' + str(type(highlight_location)))

    _svg_ = []

    # Convert to y coordinates pairs
    y_coord_pairs = []
    for k in location_lookups:
        for _span_ in location_lookups[k]:
            _poly_ = self.spanGeometry(_span_[0], _span_[1])
            x0, y0, x1, y1 = _poly_.bounds
            y_coord_pairs.append((y0, y1))
    y_coord_pairs = sorted(y_coord_pairs)

    # Consolidate
    y_consolidated_pairs, i = [], 0
    while i < len(y_coord_pairs):
        j = i + 1
        while j <  len(y_coord_pairs) and y_coord_pairs[j][0] - y_coord_pairs[j-1][1] < y_merge_threshold: j += 1
        y_consolidated_pairs.append((y_coord_pairs[i][0], y_coord_pairs[j-1][1]))
        i = j
    #for _pair_ in y_consolidated_pairs:
        #_x_ = random.randint(0, 128)
        #_svg_.append(f'<line x1="{_x_}" y1="{_pair_[0]}" x2="{_x_}" y2="{_pair_[1]}" stroke="#ff0000" stroke-width="2.0"/>')
        #_svg_.append(f'<line x1="{_x_-40}" y1="{_pair_[0]}" x2="{_x_+40}" y2="{_pair_[0]}" stroke="#ff0000" stroke-width="2.0"/>')
        #_svg_.append(f'<line x1="{_x_-40}" y1="{_pair_[1]}" x2="{_x_+40}" y2="{_pair_[1]}" stroke="#ff0000" stroke-width="2.0"/>')

    new_y_lu   = {}
    max_y_seen = 0
    max_x_rendered, max_y_rendered = 128, 128
    for _poly_ in self.geom_to_word:
        x0, y0, x1, y1 = _poly_.bounds
        max_y_seen     = max(max_y_seen, y1)
        y_mid          = (y0 + y1) / 2.0
        i              = None
        for j in range(len(y_consolidated_pairs)):
            if y_mid >= y_consolidated_pairs[j][0] and y_mid <= y_consolidated_pairs[j][1]: i = j
        if i is not None:
            rendered_so_far = 0 if y_consolidated_pairs[0][0] == 0.0 else y_render_gap
            for j in range(0, i): rendered_so_far += y_consolidated_pairs[j][1] - y_consolidated_pairs[j][0] + y_render_gap
            _y_ = y1 - 2*self.y_ins - y_consolidated_pairs[i][0] + rendered_so_far
            new_y_lu[y1] = _y_
            _svg_.append(rt.svgText(self.geom_to_word[_poly_], x0, _y_, self.txt_h))
            if x1  > max_x_rendered: max_x_rendered = x1
            if _y_ > max_y_rendered: max_y_rendered = _y_

    if y_consolidated_pairs[-1][1] != max_y_seen: max_y_rendered += y_render_gap

    for _highlight_ in location_lookups:
        _color_ = color_lookups[_highlight_]
        _spans_ = location_lookups[_highlight_]
        for _span_ in _spans_:
            _poly_ = self.spanGeometry(_span_[0], _span_[1])
            x0, y0, x1, y1 = _poly_.bounds
            closest_y = None
            for _y_ in new_y_lu:
                if   closest_y is None: closest_y = _y_
                elif abs(_y_ - y1) < abs(closest_y - y1): closest_y = _y_
            new_y = new_y_lu[closest_y]
            _poly_translated_ = affinity.translate(_poly_, 0, new_y - y1)
            _svg_.append(f'<path d="{self.rt_self.shapelyPolygonToSVGPathDescription(_poly_translated_)}" fill="{_color_}" opacity="0.2" />')

    return f'<svg x="0" y="0" width="{max_x_rendered + self.x_ins}" height="{max_y_rendered + self.y_ins}">' + ''.join(_svg_) + '</svg>'

rt.tile([compareHighlightsPixelReprs(_tb_, my_markups), 
         _tb_.highlights(all_lus)])

In [ ]:
rt.tile([compareHighlightsPixelReprs(_tb_, my_markups)])